# PART 1

The first part of this notebook implements GPU acceleration to evaluate a particle sum. We evaluate the sum $$
f(x_i) = \sum_{j} g(x_i, y_j) c_j
$$ for weights $c_j\in\mathbb{C}$, targets $x_i\in\mathbb{R}^3$ and sources $y_j\in\mathbb{R}^3$, where $g(x, y) = e^{-\frac{|x- y|^2}{2\sigma^2}}$ is the radial basis function kernel .

In [ ]:
import numpy as np
import numba
from numba import cuda
import math
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

We first define a function to generate arrays of $m$ target and $n$ source points and their assosciated weights, as well as defining the number of thread blocks $(\ell, p)$ in a grid for our GPU implementation, where $\ell = (m+15) / 16$ and $p = (n + 31) / 32$.

In [ ]:
def target_source_generate(target_points, nsources):
  """Generate arrays of target and source points"""
  plot_grid = np.mgrid[0:1:target_points * 1j, 0:1:target_points * 1j]

  #define targets, sources and weights in 3 dimensions
  targets_xy = np.vstack((plot_grid[0].ravel(),
                          plot_grid[1].ravel(),
                          np.zeros(plot_grid[0].size, dtype=np.float32))).T

  targets_xz = np.vstack((plot_grid[0].ravel(),
                          np.zeros(plot_grid[0].size,dtype=np.float32),
                          plot_grid[1].ravel())).T

  targets_yz = np.vstack((np.zeros(plot_grid[0].size, dtype=np.float32),
                        plot_grid[0].ravel(),
                        plot_grid[1].ravel())).T

  targets = np.float32(np.vstack((targets_xy, targets_xz, targets_yz)))

  #total number of targets and sources
  m=targets.shape[0]
  n=nsources

  rand = np.random.RandomState(0)

  sources = np.float32(rand.rand(n, 3))

  weights = np.float32(rand.rand(n))


  #create grid of (l,p) thread blocks
  l=(m+15)//SX
  p=(n+31)//SY

  return targets, sources, weights, n, m, l ,p

We now define a CUDA kernel to evaluate the rbf function at all target and source points. This takes as inputs the `sources`, `targets` and `weights` defined above, as well as an `intermediate_result` GPU device array of size (m, p). Each value in this array stores the evaluation of each target point over the number of source points in each thread block.

A separate CUDA kernel is then defines to sum the partial sums for each target over all thread blocks, producing `final_results`, an array of complete rbf sums for each target point.


In [ ]:
@cuda.jit
def rbf_evaluation_cuda_adapted(sources, targets, weights, intermediate_result):
  """Evaluate the rbf function sum"""

  local_result = cuda.shared.array((SX,SY), numba.float32)
  local_targets = cuda.shared.array((SX, 3), numba.float32)
  local_sources = cuda.shared.array((SY, 3), numba.float32)
  local_weights = cuda.shared.array(SY, numba.float32)

  #block position
  block_y=cuda.blockIdx.y
  block_x=cuda.blockIdx.x

  #thread pos within block
  tx = cuda.threadIdx.x
  ty = cuda.threadIdx.y
  
  
  #global thread position
  px, py = cuda.grid(2)

  if px >= m:
    return
  
  #import all data within thread block to shared memory to prevent having to read from CPU repeatedly 
  if ty == 0:
    for index in range(3):
      local_targets[tx, index] = targets[px, index]

    for index in range(SY):
            local_result[tx, index] = numba.float32(0)

  if tx == 0:
    for index in range(3):
      local_sources[ty, index] = sources[py, index]
    local_weights[ty] = weights[py]


  cuda.syncthreads()

  #compute local result for all targets and all sources in thread block
  squared_diff = numba.float32(0)
  for index in range(3):
    squared_diff += (local_targets[tx,index] - local_sources[ty,index])**2
  local_result[tx,ty] = math.exp(-squared_diff / ( numba.float32(2) * numba.float32(sigma)**2)) * local_weights[ty]

  cuda.syncthreads()

  #sum values for each target assosciated with each source  and place in intermediate_result array
  if ty==0:
    res = numba.float32(0)
    for index in range(SY):
      res += local_result[tx, index]
    intermediate_result[px, block_y] =res


@cuda.jit
def summation_kernel(intermediate_results, final_results):
  """Sum the total rbf value for each target over all threadblocks"""
  px=cuda.grid(1)
  if px >= intermediate_results.shape[0]:
    return

  threads= cuda.blockDim.x
  tx = cuda.threadIdx.x

  sum = numba.float32(0)
  for index in range(p):
    sum+=intermediate_results[px,index]
  final_results[px]=sum

We now define a parallelised Python Numba implementation of the rbf sum for comparison.

In [ ]:
@numba.njit(parallel=True)
def rbf_evaluation(sources, targets, weights, result):
    """Evaluate the RBF sum."""
    
    n = len(sources)
    m = len(targets)
        
    result[:] = 0
    for index in numba.prange(m):
        result[index] = np.sum(np.exp(-np.sum(np.abs(targets[index] - sources)**2, axis=1) / (2 * sigma**2)) * weights)

We define threadblocks of size (16x32) in a grid of size (l,p) as defined above. The choice of these threadblock dimensions ensures maximum occupancy on each 32 bit register within the GPU's streaming multiprocessors.

The arrays `intermediate_result` and `final_results` here are defined as CUDA device arrays before being parsed into their kernels. This prevents unnecessary transfer between the CPU and GPU.

Using the Python Numba implementation defined above, and by summing the resulting arrays from both implementations, we can verify that our CUDA implementation is in agreement up to (very nearly) single precission. (There may be a small error in my kernel that I'm missing).

In [ ]:
sigma = .1

SX=16
SY=32

targets, sources,weights, n, m, l ,p = target_source_generate(400,3200)
print(targets.shape)
intermediate_result_host= np.zeros((m,p), dtype=np.float32)
#intermediate_result = cuda.device_array((m,p), dtype=np.float32)
intermediate_result=cuda.to_device(intermediate_result_host)
rbf_evaluation_cuda_adapted[(l,p), (SX,SY)](sources.astype('float32'), targets.astype('float32'), weights.astype('float32'), intermediate_result)

host_intermediate_result= intermediate_result.copy_to_host()

final_results = cuda.device_array(m, dtype =np.float32)

summation_kernel[(m+31)//32,32](intermediate_result, final_results)
host_final_results= final_results.copy_to_host()

print("CUDA Result: ",np.sum(host_final_results))

#python numba implementation
result = np.zeros(len(targets), dtype=np.float32)
rbf_evaluation(sources, targets, weights, result)
print("Python Numba Result: ", np.sum(result))
  

(480000, 3)
CUDA Result:  5241298.0
Python Numba Result:  5241297.0


## Benchmarking
We now examine the effect of varying numbers of source and target points on the runtime of both our CUDA implementation and the Python Numba implementation. Firstly we define methods which include the preparation of sources and the calculation of the rbf sum arrays.

In [ ]:
def rbf_process(target_points, nsources):
  """Produce targets, sources and sum over the rbf function using the CUDA implementation"""
  targets, sources, weights, n, m, l ,p = target_source_generate(target_points,nsources)
  intermediate_result = cuda.device_array((m,p), dtype=np.float32)
  rbf_evaluation_cuda_adapted[(l,p), (SX,SY)](sources.astype('float32'), targets.astype('float32'), weights.astype('float32'), intermediate_result)
  final_results = cuda.device_array(m, dtype =np.float32)
  summation_kernel[(m+31)//32,32](intermediate_result, final_results)

In [ ]:
def numba_rbf_process(target_points, nsources):
  """Produce targets, sources and sum over the rbf function using the Python Numba implementation"""
  targets, sources, weights, n, m, l ,p = target_source_generate(target_points,nsources)
  result = np.zeros(len(targets), dtype=np.float32)
  rbf_evaluation(sources, targets, weights, result)

In [ ]:
target_points=50; nsources=50
m=target_points*target_points*3
print("n= ", nsources, ", m= ", m)
print("Cuda implementation:")
%timeit rbf_process(target_points, nsources)
print("Numba implementation:")
%timeit numba_rbf_process(target_points, nsources)

n=  50 , m=  7500
Cuda implementation:
100 loops, best of 3: 3.39 ms per loop
Numba implementation:
100 loops, best of 3: 12.5 ms per loop


In [ ]:
target_points=50; nsources=500
m=target_points*target_points*3
print("n= ", nsources, ", m= ", m)
print("Cuda implementation:")
%timeit rbf_process(target_points, nsources)
print("Numba implementation:")
%timeit numba_rbf_process(target_points, nsources)

n=  500 , m=  7500
Cuda implementation:
100 loops, best of 3: 4.51 ms per loop
Numba implementation:
10 loops, best of 3: 90.4 ms per loop


In [ ]:
target_points=50; nsources=5000
m=target_points*target_points*3
print("n= ", nsources, ", m= ", m)
print("Cuda implementation:")
%timeit rbf_process(target_points, nsources)
print("Numba implementation:")
%timeit numba_rbf_process(target_points, nsources)

n=  5000 , m=  7500
Cuda implementation:
100 loops, best of 3: 11 ms per loop
Numba implementation:
1 loop, best of 3: 882 ms per loop


In [ ]:
target_points=500; nsources=500
m=target_points*target_points*3
print("n= ", nsources, ", m= ", m)
print("Cuda implementation:")
%timeit rbf_process(target_points, nsources)
print("Numba implementation:")
%timeit numba_rbf_process(target_points, nsources)

n=  500 , m=  750000
Cuda implementation:
10 loops, best of 3: 72.8 ms per loop
Numba implementation:
1 loop, best of 3: 9 s per loop


Here we define the process required to complete the RBF sum computations on the GPU device alone, and find the time taken for various numbers of sources and targets. We see here that a large proportion of the time to run the function is taken up by simply producing the arrays of targets and sources. 

In [ ]:
def rbf_process_on_device(targets, sources, weights, n, m, l ,p):
  intermediate_result = cuda.device_array((m,p), dtype=np.float32)
  rbf_evaluation_cuda_adapted[(l,p), (SX,SY)](sources.astype('float32'), targets.astype('float32'), weights.astype('float32'), intermediate_result)
  final_results = cuda.device_array(m, dtype =np.float32)
  summation_kernel[(m+31)//32,32](intermediate_result, final_results)

In [ ]:
targets, sources, weights, n, m, l ,p = target_source_generate(50,50)
print("n= ", n, ", m= ", m)
print("Cuda implementation:")
%timeit rbf_process_on_device(targets, sources, weights, n, m, l ,p)
result = np.zeros(len(targets), dtype=np.float32)
print("Numba implementation:")
%timeit rbf_evaluation(sources, targets, weights, result)

n=  50 , m=  7500
Cuda implementation:
100 loops, best of 3: 2.8 ms per loop
Numba implementation:
100 loops, best of 3: 12.1 ms per loop


In [ ]:
targets, sources, weights, n, m, l ,p = target_source_generate(50,500)
print("n= ", n, ", m= ", m)
print("Cuda implementation:")
%timeit rbf_process_on_device(targets, sources, weights, n, m, l ,p)
result = np.zeros(len(targets), dtype=np.float32)
print("Numba implementation:")
%timeit rbf_evaluation(sources, targets, weights, result)

n=  500 , m=  7500
Cuda implementation:
100 loops, best of 3: 3.82 ms per loop
Numba implementation:
10 loops, best of 3: 90.5 ms per loop


In [ ]:
targets, sources, weights, n, m, l ,p = target_source_generate(5000,1000)
print("n= ", n, ", m= ", m)
print("Cuda implementation:")
%timeit rbf_process_on_device(targets, sources, weights, n, m, l ,p)
result = np.zeros(len(targets), dtype=np.float32)
# print("Numba implementation:")
# %timeit rbf_evaluation(sources, targets, weights, result)

n=  1000 , m=  75000000
Cuda implementation:
1 loop, best of 3: 3.59 s per loop


### Memory Transfer

We now examine the times for memory transfers from CPU to GPU and back for fixed `target_points`=50 and increasing numbers of sources. We observe that these values are significant compared to the times taken for functions defined above. Though copying arrays from the GPU device to the CPU is consistently faster than copying from CPU to GPU, both would cause significant slowdown if arrays in a GPU kernel were repeatedly copied between GPU and CPU.

In [ ]:
targets, sources1, weights, n, m, l ,p = target_source_generate(50,500)
print("Number of Sources = ", len(sources1))
%timeit sources_device1 = cuda.to_device(sources1)
targets, sources2, weights, n, m, l ,p = target_source_generate(50,5000)
print("Number of Sources = ", len(sources2))
%timeit sources_device2 = cuda.to_device(sources2)

targets, sources3, weights, n, m, l ,p = target_source_generate(50,50000)
print("Number of Sources = ", len(sources3))
%timeit sources_device3 = cuda.to_device(sources3)

targets, sources4, weights, n, m, l ,p = target_source_generate(50,500000)
print("Number of Sources = ", len(sources4))
%timeit sources_device4 = cuda.to_device(sources4)


Number of Sources =  500
The slowest run took 1836.74 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 375 µs per loop
Number of Sources =  5000
1000 loops, best of 3: 398 µs per loop
Number of Sources =  50000
1000 loops, best of 3: 753 µs per loop
Number of Sources =  500000
1000 loops, best of 3: 1.7 ms per loop


In [ ]:
sources_device1 = cuda.to_device(sources1)
sources_device2 = cuda.to_device(sources2)
sources_device3 = cuda.to_device(sources3)
sources_device4 = cuda.to_device(sources4)

print("Number of Sources = ", len(sources1))
%timeit sources_device1.copy_to_host()
print("Number of Sources = ", len(sources2))
%timeit sources_device2.copy_to_host()
print("Number of Sources = ", len(sources3))
%timeit sources_device3.copy_to_host()
print("Number of Sources = ", len(sources4))
%timeit sources_device4.copy_to_host()

Number of Sources =  500
The slowest run took 11.19 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 65.1 µs per loop
Number of Sources =  5000
The slowest run took 9.88 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 79.2 µs per loop
Number of Sources =  50000
1000 loops, best of 3: 282 µs per loop
Number of Sources =  500000
1000 loops, best of 3: 1.23 ms per loop


# PART 2


We now move on to analyse the Poisson problem$$
-\Delta u = 1
$$ on the unit square $\Omega = [0, 1]^2$ with $x = 0$ on $\partial\Omega$. This problem can be set up as a discrete linear matrix problem $ A x = f $ if we discretise points $(x_i, y_j)$ in the unit square such that $x_i = ih$, $y_j = jh$ where $h = \frac{1}{N - 1}$ such that each point $u(x_i, y_j)$ has value $u_{i,j}$. $u_{i,j}$ can then be represented as a 1D vector $u_{i, j} = x_{jN + i}$.

The LHS of the linear matrix problem can be approximated as $$
-\Delta u_{i, j}\approx \frac{4u_{i, j} - u_{i - 1, j} - u_{i + 1, j} - u_{i, j - 1} - u_{i, j+ 1}}{h^2}.
$$

Below we define a GPU accelerated kernel to compute the LHS matrix-vector product without the explicit creation of the matrix $A$. This kernel is then compared to a non GPU accelerated method which explicitly creates a sparse matrix representing $A$ in order to solve the linear matrix problem.

In [ ]:
@cuda.jit
def poisson_kernel(x,device_f):

  """if thread is boundary value- write corresponding u_i,j value in reslut array
  if thread is assoc with interior point- evaluate 5 point stencil for corresponding interior point"""

  thread=cuda.blockIdx.x

  if thread > x.shape[0]:
    return

  #the associated row and column in 2d array u_i,j for each value in the 1d array 'x'
  i = thread // N
  j = thread - (i * N)
  
  #boundary condition
  if i == 0  or i==N-1 or j==0 or j==N-1:
    device_f[thread] = x[thread]
  #five-point stencil
  else:
    device_f[thread] = ( 4*x[thread] - x[thread-1] - x[thread+1] - x[thread-N] - x[thread+N])/(h*h)
  cuda.syncthreads()


In [ ]:


def discretise_poisson(N):
    """Generate the matrix and rhs associated with the discrete Poisson operator."""
    
    nelements = 5 * N**2 - 16 * N + 16
    
    row_ind = np.empty(nelements, dtype=np.float64)
    col_ind = np.empty(nelements, dtype=np.float64)
    data = np.empty(nelements, dtype=np.float64)
    
    f = np.empty(N * N, dtype=np.float64)
    
    count = 0
    for j in range(N):
        for i in range(N):
            if i == 0 or i == N - 1 or j == 0 or j == N - 1:
                row_ind[count] = col_ind[count] = j * N + i
                data[count] =  1
                f[j * N + i] = 0
                count += 1
                
            else:
                row_ind[count : count + 5] = j * N + i
                col_ind[count] = j * N + i
                col_ind[count + 1] = j * N + i + 1
                col_ind[count + 2] = j * N + i - 1
                col_ind[count + 3] = (j + 1) * N + i
                col_ind[count + 4] = (j - 1) * N + i
                                
                data[count] = 4 * (N - 1)**2
                data[count + 1 : count + 5] = - (N - 1)**2
                f[j * N + i] = 1
                
                count += 5                                 
    return coo_matrix((data, (row_ind, col_ind)), shape=(N**2, N**2)).tocsr(), f

We now define a Numba parrallelised function to compute the matrix-vector product of a CSR matrix and a vector.

In [ ]:
@numba.jit(nopython=True, parallel=True)
def csr_matvec(data, indices, indptr, shape, x):
    """Evaluates the matrix-vector product with a CSR matrix."""
    # Get the rows and columns
    
    m, n = shape
    
    y = np.zeros(m, dtype=np.float64)
        
    for row_index in numba.prange(m):
        col_start = indptr[row_index]
        col_end = indptr[row_index + 1]
        for col_index in range(col_start, col_end):
            y[row_index] += data[col_index] * x[indices[col_index]]
            
    return y

Having defined our two methods for computing the result of the matrix-vector multiplication, we can set up a distribution of $u_{i,j}$ values and transform into a 1D vector `x`. We can then explicitly compute the matrix-vector product $Ax$ using the non-GPU accelerated function and compare to our CUDA function.

We find here that the error between the result matrices produced by the two functions is zero, validating our CUDA accelerated method.

In [ ]:
N=100
h=1/(N-1)
Nsq=N*N

#set up initial distribution u, convert to 1darray x
u=np.zeros((N,N))

for n in range (1,int(N/2 +1/2)+1):
    u[n:(N-1)-(n-1), n:(N-1)-(n-1)]=n*n

x=np.zeros(N*N, dtype=np.float32)
for i in range(N):
  for j in range(N):
    x[j * N + i] =u[i,j]


#sparse matrix
A, f = discretise_poisson(N)
sparse_f =  csr_matvec(A.data, A.indices, A.indptr, A.shape, x)

#cuda implementation
device_x = cuda.to_device(x)
device_f = cuda.device_array((N*N,), dtype=np.float32)
poisson_kernel[Nsq,1](device_x,device_f)
host_f = device_f.copy_to_host()


rel_error = np.linalg.norm(sparse_f - host_f, np.inf) / np.linalg.norm(host_f, np.inf)
print(f"Error: {round(rel_error, 2)}.")

Error: 0.0.


In [ ]:
sol = spsolve(A, f)
y = csr_matvec(A.data, A.indices, A.indptr, A.shape, sol)

device_sol = cuda.to_device(sol)
device_y = cuda.device_array((N*N,), dtype=np.float32)
poisson_kernel[Nsq,1](device_sol,device_y)
host_y = device_y.copy_to_host()

rel_error = np.linalg.norm(y - host_y, np.inf) / np.linalg.norm(host_y, np.inf)
print(f"Error: {round(rel_error, 2)}.")

Error: 0.0.


## Benchmarking

Here we set up a loop over values of matrix dimension $N$, to find the time taken for our CUDA accelerated function to compute the result array. Time could be saved here by splitting the values in `x` into threadblocks of a greater number of threads and loading these values into the shared memory for each block, rather than accessing the global memory to find each data point in the five-point stencil as is currently the case.

In [ ]:
N_array= np.linspace(10,3000,10).astype(np.int32)

for N in N_array:
  h=1/(N-1)
  Nsq=N*N

  #set up initial distribution u, convert to 1darray x
  u=np.zeros((N,N))

  for n in range (1,int(N/2 +1/2)+1):
      u[n:(N-1)-(n-1), n:(N-1)-(n-1)]=n*n

  x=np.zeros(N*N, dtype=np.float32)
  for i in range(N):
    for j in range(N):
      x[j * N + i] =u[i,j]

  device_f = cuda.device_array((N*N,), dtype=np.float32)
  print("N=",N,":")
  %timeit -n 50 poisson_kernel[Nsq,1](x,device_f)


N= 10 :
50 loops, best of 3: 826 µs per loop
N= 342 :
50 loops, best of 3: 2.46 ms per loop
N= 674 :
50 loops, best of 3: 5.85 ms per loop
N= 1006 :
50 loops, best of 3: 11.6 ms per loop
N= 1338 :
50 loops, best of 3: 19.7 ms per loop
N= 1671 :
50 loops, best of 3: 30.4 ms per loop
N= 2003 :
50 loops, best of 3: 43.1 ms per loop
N= 2335 :
50 loops, best of 3: 58.2 ms per loop
N= 2667 :
50 loops, best of 3: 75.4 ms per loop
N= 3000 :
50 loops, best of 3: 95.2 ms per loop
